# Working With Algorithms

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/developmentseed/titiler/main?filepath=docs%2Fexamples%2F%2Fnotebooks%2FWorking_with_Algorithm.ipynb)

For this demo we will use some elevation data from https://www.swisstopo.admin.ch/fr/geodata/height/alti3d.html  dataset

#### Requirements
- folium
- httpx

`!pip install folium httpx`

In [1]:
import json
import httpx

from folium import Map, TileLayer

In [2]:
titiler_endpoint = "https://titiler.xyz"  # Developmentseed Demo endpoint. Please be kind.

In [3]:
url = "https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2573-1085/swissalti3d_2019_2573-1085_0.5_2056_5728.tif"

## Get COG Info

In [4]:
# Fetch dataset Metadata
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

print(r)

{'bounds': [7.090624928537461, 45.91605844102821, 7.1035698381384185, 45.92509300025415], 'minzoom': 15, 'maxzoom': 18, 'band_metadata': [['b1', {'STATISTICS_COVARIANCES': '10685.98787505646', 'STATISTICS_EXCLUDEDVALUES': '-9999', 'STATISTICS_MAXIMUM': '2015.0944824219', 'STATISTICS_MEAN': '1754.471184271', 'STATISTICS_MINIMUM': '1615.8128662109', 'STATISTICS_SKIPFACTORX': '1', 'STATISTICS_SKIPFACTORY': '1', 'STATISTICS_STDDEV': '103.37305197708'}]], 'band_descriptions': [['b1', '']], 'dtype': 'float32', 'nodata_type': 'Nodata', 'colorinterp': ['gray'], 'driver': 'GTiff', 'count': 1, 'width': 2000, 'height': 2000, 'overviews': [2, 4, 8], 'nodata_value': -9999.0}


### Display Tiles

By default, the tiles will be rescaled from min/max from dataset statistics (1615.812 / 2015.09448)

In [5]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Office fédéral de topographie swisstopo"
).add_to(m)
m

In [8]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        # rio-tiler cannot rescale automatically the data when using a colormap
        "rescale": "1615.812,2015.09448",
        "colormap_name": "terrain",
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Office fédéral de topographie swisstopo"
)
aod_layer.add_to(m)
m

## Show Available Algorithms

In [9]:
# Fetch algorithms
print("Available algorithm")
print(list(httpx.get(f"{titiler_endpoint}/algorithms").json()))
print()
print("Metadata from `Hillshade` algorithm")
meta = httpx.get(f"{titiler_endpoint}/algorithms/hillshade").json()
print("Inputs")
print(meta["inputs"])
print("Outputs")
print(meta["outputs"])
print("Parameters")
print(meta["parameters"])

Available algorithm
['hillshade', 'contours', 'normalizedIndex', 'terrarium', 'terrainrgb']

Metadata from `Hillshade` algorithm
Inputs
{'nbands': 1}
Outputs
{'nbands': 1, 'dtype': 'uint8', 'min': None, 'max': None}
Parameters
{'azimuth': {'default': 90, 'title': 'Azimuth', 'type': 'integer'}, 'angle_altitude': {'default': 90.0, 'title': 'Angle Altitude', 'type': 'number'}, 'buffer': {'default': 3, 'title': 'Buffer', 'type': 'integer'}}


## Display Hillshade Tiles

In [10]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "algorithm": "hillshade",
        # Hillshade algorithm use a 3pixel buffer so we need 
        # to tell the tiler to apply a 3 pixel buffer around each tile
        "buffer": 3,
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
)
aod_layer.add_to(m)
m

### Pass parameters to the algorithm

In [13]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "algorithm": "contours",
        "algorithm_params": json.dumps(
            {
                "increment": 20, # contour line every 20 meters
                "thickness": 2, # 2m thickness
                "minz": 1600,
                "maxz": 2000
            }
        ),
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
).add_to(m)
m

In [12]:
r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "algorithm": "contours",
        "algorithm_params": json.dumps(
            {
                "increment": 5, # contour line every 5 meters
                "thickness": 1, # 1m thickness
                "minz": 1600,
                "maxz": 2000
            }
        ),
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
).add_to(m)
m